## Interpolation Using OpeCV

In [1]:
# example 1
import numpy as np
import cv2

data = np.array([[1, 5],
                 [4, 8]])

# new dimension
width, height = 2, 2
upscale = 2
new_dim = (width*upscale, height*upscale)

# upsampling the data
resize_nearest = cv2.resize(data, new_dim, 
                            interpolation=cv2.INTER_NEAREST)

print(data)
print(resize_nearest)

[[1 5]
 [4 8]]
[[1 1 5 5]
 [1 1 5 5]
 [4 4 8 8]
 [4 4 8 8]]


In [2]:
# example 2
import numpy as np
import cv2

data = np.array([[1.0, 5.0],
                 [4.0, 8.0]])

# new dimension
width, height = 2, 2
upscale = 2
new_dim = (width*upscale, height*upscale)

# upsampling the data
resize_bilinear = cv2.resize(data, new_dim, 
                             interpolation=cv2.INTER_LINEAR)

print(data)
print(resize_bilinear)

[[1. 5.]
 [4. 8.]]
[[1.   2.   4.   5.  ]
 [1.75 2.75 4.75 5.75]
 [3.25 4.25 6.25 7.25]
 [4.   5.   7.   8.  ]]


## Interpolation Using Pytorch

In [8]:
import numpy as np
import torch
import torch.nn as nn

data = torch.Tensor([[1.0, 5.0],
                     [4.0, 8.0]])
data = data.reshape(1, 1, 2, 2)   # bs, channel, H, W
print(data.shape)

torch.Size([1, 1, 2, 2])


In [9]:
# nearest
up_sample = nn.Upsample(scale_factor=2,
                        mode='nearest')
output = up_sample(data)
print(output)

tensor([[[[1., 1., 5., 5.],
          [1., 1., 5., 5.],
          [4., 4., 8., 8.],
          [4., 4., 8., 8.]]]])


In [10]:
# bilinear
up_sample = nn.Upsample(scale_factor=2,
                        mode='bilinear', 
                        align_corners=True)
output = up_sample(data)
print(output)

tensor([[[[1.0000, 2.3333, 3.6667, 5.0000],
          [2.0000, 3.3333, 4.6667, 6.0000],
          [3.0000, 4.3333, 5.6667, 7.0000],
          [4.0000, 5.3333, 6.6667, 8.0000]]]])


## Using Reshape() Function

In [15]:
import numpy as np
import torch
import torch.nn as nn

data = torch.Tensor([[1.0, 5.0],
                     [4.0, 8.0]])
data = data.reshape(1, 1, 2, 2)   # bs, channel, H, W
print(data.shape)

torch.Size([1, 1, 2, 2])


In [18]:
C_in, H_in, W_in = 1, 2, 2
c_out = 1
conv = nn.Conv2d(C_in, c_out*4, kernel_size=3, padding=1)

output = conv(data)
output = output.reshape(1, c_out, H_in*2, W_in*2)
print(output.shape)

torch.Size([1, 1, 4, 4])


In [19]:
C_in, H_in, W_in = 1, 2, 2
c_out = 1
conv = nn.Conv2d(C_in, c_out*4, kernel_size=3, padding=1)

output = conv(data)
output = output.view(1, c_out, H_in*2, W_in*2)
print(output.shape)

torch.Size([1, 1, 4, 4])


## PixelShuffle

In [28]:
import torch
import torch.nn as nn

data = torch.Tensor([1, 2, 3, 4])
data = data.reshape(1, 4, 1, 1)   # bs, channel, H, W
print(data.shape)

torch.Size([1, 4, 1, 1])


In [29]:
shuffle = nn.PixelShuffle(2)
output = shuffle(data)
print(output)
print(output.shape)

tensor([[[[1., 2.],
          [3., 4.]]]])
torch.Size([1, 1, 2, 2])


In [31]:
import torch
import torch.nn as nn

data = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8])
data = data.reshape(1, 8, 1, 1)   # bs, channel, H, W

shuffle = nn.PixelShuffle(2)
output = shuffle(data)
print(output.shape)

torch.Size([1, 2, 2, 2])


In [32]:
print(output[0, 0, :, :])

tensor([[1., 2.],
        [3., 4.]])


In [33]:
print(output[0, 1, :, :])

tensor([[5., 6.],
        [7., 8.]])


## Unpooling

In [35]:
import torch
import torch.nn as nn

data = torch.Tensor([[1, 2, 3, 4],
                     [5, 6, 7, 8],
                     [9, 8, 7, 6],
                     [5, 4, 3, 2]])
data = data.reshape(1, 1, 4, 4)   # bs, channel, H, W
print(data.shape)
print(data)

torch.Size([1, 1, 4, 4])
tensor([[[[1., 2., 3., 4.],
          [5., 6., 7., 8.],
          [9., 8., 7., 6.],
          [5., 4., 3., 2.]]]])


In [37]:
max_pooling = nn.MaxPool2d(2, return_indices=True)
output_mp, indices = max_pooling(data)

print(output_mp)
print(indices)

tensor([[[[6., 8.],
          [9., 7.]]]])
tensor([[[[ 5,  7],
          [ 8, 10]]]])


In [43]:
indices.shape

torch.Size([1, 1, 2, 2])

In [38]:
max_unpooling = nn.MaxUnpool2d(kernel_size=2)
output_mup = max_unpooling(output_mp, indices)
print(output_mup.shape)

torch.Size([1, 1, 4, 4])


In [39]:
print(output_mup[0,0,:,:])

tensor([[0., 0., 0., 0.],
        [0., 6., 0., 8.],
        [9., 0., 7., 0.],
        [0., 0., 0., 0.]])


## ConvTranspose

In [1]:
import numpy as np
import torch
import torch.nn as nn

data = torch.Tensor([[1.0, 5.0],
                     [4.0, 8.0]])
data = data.reshape(1, 1, 2, 2)   # bs, channel, H, W
print(data.shape)

torch.Size([1, 1, 2, 2])


In [2]:
conv_t = nn.ConvTranspose2d(1, 1, kernel_size=4, stride=2, padding=1, output_padding=0)

output = conv_t(data)
print(output.shape)

torch.Size([1, 1, 4, 4])


In [3]:
conv_t = nn.ConvTranspose2d(1, 1, kernel_size=3, stride=2, padding=0, output_padding=0)

output = conv_t(data)
print(output.shape)

torch.Size([1, 1, 5, 5])


In [57]:
conv_t = nn.ConvTranspose2d(1, 1, kernel_size=4, stride=2, padding=0, output_padding=1)

output = conv_t(data)
print(output.shape)

torch.Size([1, 1, 7, 7])


In [59]:
conv_t = nn.ConvTranspose2d(1, 1, kernel_size=4, stride=2, padding=1, output_padding=0)

output = conv_t(data)
print(output.shape)

torch.Size([1, 1, 4, 4])
